# Modelo para predição de tempo de julgamento de processos judicais

O modelo será teinado com base nos dados disponíveis no Datajud, que é a base nacional de metadados processuais do poder judiciário brasileiro.

[Documentação do Datajud](https://datajud-wiki.cnj.jus.br/)

Os dados disponíveis são:

Tipo do Tribunal (Estadual, Federal, Trabalho, Eleitoal)
Estado
Grau do processo (G1, G2, TR, JE)
Unidade Judicial / Vara
Classe Processual
Assunto
Data Ajuizamento(Data de distribuição do processo)
Lista de movimentações

As classes, assuntos e movimentações eguem uma tabela nacional, com seus códigos e classificação previamente estabelecida.
Essa padronização é a TPU - [Tabelas Processuais Unificadas](https://www.cnj.jus.br/sgt/consulta_publica_classes.php)


## SQL extração classes em seus níveis

WITH nivel_1 AS (
	SELECT clcodigo AS codnivel1 FROM bpu.classe WHERE clcodpai IS NULL AND clcodigo < 1000000
),nivel_2 AS (
	SELECT codnivel1, clcodigo AS codnivel2 FROM nivel_1 LEFT JOIN bpu.classe ON clcodpai = codnivel1
	UNION
	SELECT codnivel1, NULL AS codnivel2 FROM nivel_1
),
nivel_3 AS (
	SELECT codnivel1, codnivel2, clcodigo AS codnivel3 FROM nivel_2 LEFT JOIN bpu.classe ON clcodpai = codnivel2
	UNION
	SELECT codnivel1, codnivel2, NULL AS codnivel3 FROM nivel_2
),
nivel_4 AS (
	SELECT codnivel1, codnivel2, codnivel3, clcodigo AS codnivel4 FROM nivel_3 LEFT JOIN bpu.classe ON clcodpai = codnivel3
	UNION
	SELECT codnivel1, codnivel2, codnivel3, NULL AS codnivel4 FROM nivel_3
),
nivel_5 AS (
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, clcodigo AS codnivel5 FROM nivel_4 LEFT JOIN bpu.classe ON clcodpai = codnivel4
	UNION
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, NULL AS codnivel5 FROM nivel_4
),
nivel_6 AS (
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, codnivel5, clcodigo AS codnivel6 FROM nivel_5 LEFT JOIN bpu.classe ON clcodpai = codnivel5
	UNION
	SELECT codnivel1, codnivel2, codnivel3, codnivel4, codnivel5, NULL AS codnivel6 FROM nivel_5
)
SELECT
	codnivel1,
	codnivel2,
	codnivel3,
	codnivel4,
	codnivel5,
	COALESCE(codnivel6,codnivel5,codnivel4,codnivel3,codnivel2,codnivel1) AS codclasse,
	cldescricao AS descclasse
FROM nivel_6
	LEFT JOIN bpu.classe ON COALESCE(codnivel6,codnivel5,codnivel4,codnivel3,codnivel2,codnivel1)=clcodigo
ORDER BY codnivel1,codnivel2,codnivel3,codnivel4,codnivel5,codclasse;

In [1]:
import pandas as pd
import requests
import json

In [2]:
classes_tpu = pd.read_csv('https://drive.google.com/uc?export=download&id=19Y7Zjwo8bZFHZ6rwO_fMsVljU0frLAon')
assuntos_tpu = pd.read_csv('https://drive.google.com/uc?export=download&id=17FtEye4Vq75cHUjvhiC4Rl5oKlCPpudK', sep=',')

In [3]:
mov_julgamentos = [193,196,198,200,202,208,210,212,214,218,219,220,221,228,230,235,236,237,238,239,240,241,242,244,385,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,471,472,473,853,871,884,900,901,972,973,1042,1043,1044,1045,1046,1047,1048,1049,1050,10953,10961,10964,10965,11373,11374,11375,11376,11377,11378,11379,11380,11381,11394,11396,11401,11402,11403,11404,11405,11406,11407,11408,11409,11411,11795,11796,11801,11876,11877,11878,11879,12028,12032,12033,12034,12041,12184,12187,12252,12253,12254,12256,12257,12258,12298,12319,12321,12322,12323,12324,12325,12326,12327,12328,12329,12330,12331,12433,12434,12435,12436,12437,12438,12439,12440,12441,12442,12443,12450,12451,12452,12453,12458,12459,12475,12615,12616,12617,12649,12650,12651,12652,12653,12654,12660,12661,12662,12663,12664,12665,12666,12667,12668,12669,12670,12671,12672,12673,12674,12675,12676,12677,12678,12679,12680,12681,12682,12683,12684,12685,12686,12687,12688,12689,12690,12691,12692,12693,12694,12695,12696,12697,12698,12699,12700,12701,12702,12703,12704,12705,12706,12707,12708,12709,12710,12711,12712,12713,12714,12715,12716,12717,12718,12719,12720,12721,12722,12723,12724,12735,12738,12792,14099,14210,14211,14213,14214,14215,14216,14217,14218,14219,14680,14777,14778,14848,14937,15022,15023,15024,15026,15027,15028,15029,15030,15165,15166,15211,15212,15213,15214,15245,15249,15250,15251,15252,15253,15254,15255,15256,15257,15258,15259,15260,15261,15262,15263,15264,15265,15266,15322,15408,1002013,1003301,1003302,1050009,1050080,1050083,1050147,4050028,4050079,4050080,4050083]

In [4]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjrr/_search"

payload = json.dumps({
    "query": {
        "bool": {
            "must": [
                {"match": {"classe.codigo": 7}}
            ]
        }
    }
}
)

#Substituir <API Key> pela Chave Pública
headers = {
  'Authorization': 'ApiKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [5]:
resposta  = json.loads(response.text)
lista = resposta['hits']['hits']
df = pd.DataFrame.from_records(lista)

In [6]:
df = df['_source'].apply(pd.Series)
df[['codigo_classe','nome_classe']] = df['classe'].apply(pd.Series)
df[['codigo_formato','nome_formato']] = df['formato'].apply(pd.Series)
df[['orgao_codigoMunicipioIBGE','orgao_codigo','orgao_nome']] = df['orgaoJulgador'].apply(pd.Series)

In [7]:
df['dataAjuizamento'] = pd.to_datetime(df['dataAjuizamento']).dt.strftime('%Y-%m-%d')

In [8]:
def extrair_assunto(assuntos):
  return assuntos[0]['codigo']

def extrair_dcr_assunto(assuntos):
  return assuntos[0]['nome']

df['codigo_assunto'] = df['assuntos'].apply(extrair_assunto)
df['dcr_assunto'] = df['assuntos'].apply(extrair_dcr_assunto)

#*
df['dcr_assunto'] = df['dcr_assunto'].str.upper()
df['nome_classe'] = df['nome_classe'].str.upper()

In [9]:
def extrair_sentenca(movimentos):
  for movimento in movimentos:
    if movimento['codigo'] in mov_julgamentos:
      return movimento['dataHora']
  return None

df['data_julgamento'] = df['movimentos'].apply(extrair_sentenca)
df['dataJulgamento'] = pd.to_datetime(df['data_julgamento']).dt.strftime('%Y-%m-%d');

In [10]:
#*
df.drop(columns=['@timestamp','id','sistema','dataHoraUltimaAtualizacao','classe','orgaoJulgador','orgao_codigoMunicipioIBGE','orgao_nome','assuntos','data_julgamento','formato','nome_formato'],inplace=True)
df.dropna(subset=['dataJulgamento'], inplace=True)

In [11]:
df

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,codigo_classe,nome_classe,codigo_formato,orgao_codigo,codigo_assunto,dcr_assunto,dataJulgamento
0,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05
1,00743440420038230010,TJRR,G1,2003-12-09,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,5946,ICMS/ IMPOSTO SOBRE CIRCULAÇÃO DE MERCADORIAS,2005-09-30
2,00090109120018230010,TJRR,G1,2001-11-16,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,10423,ANULAÇÃO,2002-04-11
3,00034350520018230010,TJRR,G1,1999-09-20,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,10439,INDENIZAÇÃO POR DANO MATERIAL,2002-07-16
4,00035936020018230010,TJRR,G1,1997-08-13,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,5946,ICMS/ IMPOSTO SOBRE CIRCULAÇÃO DE MERCADORIAS,2009-06-04
5,00002817920028230030,TJRR,G1,1999-09-13,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2782,10448,PROPRIEDADE,2004-10-29
6,00714401120038230010,TJRR,G1,2003-10-08,"[{'complementosTabelados': [{'codigo': 15, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,10439,INDENIZAÇÃO POR DANO MATERIAL,2004-09-30
7,01659455220078230010,TJRR,G1,2007-07-23,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,10307,REVISÃO GERAL ANUAL (MORA DO EXECUTIVO - INCIS...,2008-10-21
8,01545766120078230010,TJRR,G1,2007-01-18,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,10307,REVISÃO GERAL ANUAL (MORA DO EXECUTIVO - INCIS...,2007-08-07
9,01550122020078230010,TJRR,G1,2007-01-30,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,10307,REVISÃO GERAL ANUAL (MORA DO EXECUTIVO - INCIS...,2007-09-13


In [12]:
assuntos = pd.read_csv('assuntos-tpu.csv')
classes = pd.read_csv('tpu-classes.csv')

In [13]:
classes = classes.rename(columns={'codnivel1': 'c_codnivel1', 'codnivel2': 'c_codnivel2', 'codnivel3': 'c_codnivel3', 'codnivel4': 'c_codnivel4', 'codnivel5':'c_codnivel5', 'codclasse':'c_codclasse'})

In [14]:
df_final = pd.merge(df, assuntos, how='left', left_on='dcr_assunto', right_on='descassunto')

In [15]:
df_final

,numeroProcesso,tribunal,grau,dataAjuizamento,movimentos,nivelSigilo,codigo_classe,nome_classe,codigo_formato,orgao_codigo,codigo_assunto,dcr_assunto,dataJulgamento,codnivel1,codnivel2,codnivel3,codnivel4,codnivel5,codassunto,descassunto
0,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05,864,2567.0,13195.0,NaN,NaN,13195,INDENIZAÇÃO POR DANO MORAL
1,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05,864,12936.0,14007.0,14010.0,NaN,14010,INDENIZAÇÃO POR DANO MORAL
2,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05,864,12936.0,14031.0,14033.0,NaN,14033,INDENIZAÇÃO POR DANO MORAL
3,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05,899,10431.0,10433.0,NaN,NaN,10433,INDENIZAÇÃO POR DANO MORAL
4,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05,1156,6220.0,7779.0,NaN,NaN,7779,INDENIZAÇÃO POR DANO MORAL
5,00784758520048230010,TJRR,G1,2004-02-11,"[{'complementosTabelados': [{'codigo': 4, 'val...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,9992,INDENIZAÇÃO POR DANO MORAL,2004-11-05,9985,9991.0,9992.0,NaN,NaN,9992,INDENIZAÇÃO POR DANO MORAL
6,00743440420038230010,TJRR,G1,2003-12-09,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2764,5946,ICMS/ IMPOSTO SOBRE CIRCULAÇÃO DE MERCADORIAS,2005-09-30,14,5916.0,5946.0,NaN,NaN,5946,ICMS/ IMPOSTO SOBRE CIRCULAÇÃO DE MERCADORIAS
7,00090109120018230010,TJRR,G1,2001-11-16,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,10423,ANULAÇÃO,2002-04-11,864,1695.0,4438.0,NaN,NaN,4438,ANULAÇÃO
8,00090109120018230010,TJRR,G1,2001-11-16,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,10423,ANULAÇÃO,2002-04-11,899,7681.0,4949.0,4951.0,NaN,4951,ANULAÇÃO
9,00090109120018230010,TJRR,G1,2001-11-16,"[{'complementosTabelados': [{'codigo': 18, 'va...",0,7,PROCEDIMENTO COMUM CÍVEL,1,2770,10423,ANULAÇÃO,2002-04-11,9985,10370.0,10382.0,NaN,NaN,10382,ANULAÇÃO


In [16]:
df_final = pd.merge(df_final, classes, how='left', left_on='nome_classe', right_on='descclasse')

In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   numeroProcesso   30 non-null     object 
 1   tribunal         30 non-null     object 
 2   grau             30 non-null     object 
 3   dataAjuizamento  30 non-null     object 
 4   movimentos       30 non-null     object 
 5   nivelSigilo      30 non-null     int64  
 6   codigo_classe    30 non-null     int64  
 7   nome_classe      30 non-null     object 
 8   codigo_formato   30 non-null     int64  
 9   orgao_codigo     30 non-null     int64  
 10  codigo_assunto   30 non-null     int64  
 11  dcr_assunto      30 non-null     object 
 12  dataJulgamento   30 non-null     object 
 13  codnivel1        30 non-null     int64  
 14  codnivel2        30 non-null     float64
 15  codnivel3        30 non-null     float64
 16  codnivel4        10 non-null     float64
 17  codnivel5        0

In [18]:
df_final.drop(columns=['movimentos','descclasse','descassunto','nome_classe', 'dcr_assunto'])

,numeroProcesso,tribunal,grau,dataAjuizamento,nivelSigilo,codigo_classe,codigo_formato,orgao_codigo,codigo_assunto,dataJulgamento,...,codnivel3,codnivel4,codnivel5,codassunto,c_codnivel1,c_codnivel2,c_codnivel3,c_codnivel4,c_codnivel5,c_codclasse
0,00784758520048230010,TJRR,G1,2004-02-11,0,7,1,2770,9992,2004-11-05,...,13195.0,NaN,NaN,13195,2,1106.0,1107.0,7.0,NaN,7
1,00784758520048230010,TJRR,G1,2004-02-11,0,7,1,2770,9992,2004-11-05,...,14007.0,14010.0,NaN,14010,2,1106.0,1107.0,7.0,NaN,7
2,00784758520048230010,TJRR,G1,2004-02-11,0,7,1,2770,9992,2004-11-05,...,14031.0,14033.0,NaN,14033,2,1106.0,1107.0,7.0,NaN,7
3,00784758520048230010,TJRR,G1,2004-02-11,0,7,1,2770,9992,2004-11-05,...,10433.0,NaN,NaN,10433,2,1106.0,1107.0,7.0,NaN,7
4,00784758520048230010,TJRR,G1,2004-02-11,0,7,1,2770,9992,2004-11-05,...,7779.0,NaN,NaN,7779,2,1106.0,1107.0,7.0,NaN,7
5,00784758520048230010,TJRR,G1,2004-02-11,0,7,1,2770,9992,2004-11-05,...,9992.0,NaN,NaN,9992,2,1106.0,1107.0,7.0,NaN,7
6,00743440420038230010,TJRR,G1,2003-12-09,0,7,1,2764,5946,2005-09-30,...,5946.0,NaN,NaN,5946,2,1106.0,1107.0,7.0,NaN,7
7,00090109120018230010,TJRR,G1,2001-11-16,0,7,1,2770,10423,2002-04-11,...,4438.0,NaN,NaN,4438,2,1106.0,1107.0,7.0,NaN,7
8,00090109120018230010,TJRR,G1,2001-11-16,0,7,1,2770,10423,2002-04-11,...,4949.0,4951.0,NaN,4951,2,1106.0,1107.0,7.0,NaN,7
9,00090109120018230010,TJRR,G1,2001-11-16,0,7,1,2770,10423,2002-04-11,...,10382.0,NaN,NaN,10382,2,1106.0,1107.0,7.0,NaN,7
